<p> This notebook is to create the XML files using the annotations already provided while downloading the dataset</p>
<p> the dataset is from : https://universe.roboflow.com/roboflow-100/bone-fracture-7fylg/dataset/2</p>

In [83]:
# import libraries
import pandas as pd

In [84]:
# function to replace dynamic parts with the given data from the excel sheet
def search_and_replace(file_path, search_word, replace_word):
   with open(file_path, 'r') as file:
      file_contents = file.read()

      updated_contents = file_contents.replace(search_word, replace_word)

   with open(file_path, 'w') as file:
      file.write(updated_contents)
   
   file.close()

In [93]:
# open the annotations excel file to extract the data from it
path = input("Enter path of the file : ")
annotations = pd.read_csv(path)
df = pd.DataFrame(annotations)
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,192_jpg.rf.a44d116b12d2efa63b37fe6bc7b5318d.jpg,837,1024,fracture,204,444,256,494
1,29_jpg.rf.a358d0249bf4ce4ecc85be891f7d1721.jpg,448,1024,fracture,243,618,270,645
2,73_jpg.rf.aa734b5afb647f0f8564eac868d4847f.jpg,383,1024,line,178,497,229,546
3,73_jpg.rf.aa734b5afb647f0f8564eac868d4847f.jpg,383,1024,angle,126,447,176,519
4,177_jpg.rf.a5f6e20d94ccf5454db6408b2d8dc0a0.jpg,603,1024,fracture,232,373,283,407


In [94]:
df.columns

Index(['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'], dtype='object')

In [95]:
# template for xml annotations
file = open('annot_template.txt', 'w')
file.write("""
<annotation>
	<folder>fracture-detection</folder>
	<filename>IMAGE_NAME</filename>
	<path>ANNOT_PATH</path>
	<source>
		<database>Unspecified</database>
	</source>
	<size>
		<width>WIDTH</width>
		<height>HEIGHT</height>
		<depth>3</depth>
	</size>
	<object>
		<name>LABEL</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>XMIN</xmin>
			<ymin>YMIN</ymin>
			<xmax>XMAX</xmax>
			<ymax>YMAX</ymax>
		</bndbox>
	</object>
</annotation>
""")
file.close()


In [96]:
# creating the xml annotations
length = len(df['filename'])

path = input("Enter Destination path: ")
for i in range(length):
    fname = df.iloc[i][0][:-4]
    fname = fname+".xml"

    f1 = open("annot_template.txt", "r")
    lines = f1.readlines()
    #print(lines)

    f2_name = "{}/{}".format(path, fname)
    f2 = open(f2_name, "w")
    for line in lines:
        f2.write(line)

    values = dict()
    values['IMAGE_NAME'] = df.iloc[i][0]
    values['ANNOT_PATH'] = '/{}/{}'.format(path, fname)
    values['WIDTH'] = df.iloc[i][1]
    values['HEIGHT'] = df.iloc[i][2]
    values['LABEL'] = df.iloc[i][3]
    values['XMIN'] = df.iloc[i][4]
    values['YMIN'] = df.iloc[i][5]
    values['XMAX'] = df.iloc[i][6]
    values['YMAX'] = df.iloc[i][7]

    f2.close()

    for k in values.keys():
        location = values['ANNOT_PATH']
        #print(location)
        #print("{} \n{} \n{} \n\n".format(location[1:], k, str(values[k])))
        search_and_replace(location[1:], k, str(values[k]))

    f1.close()

KeyboardInterrupt: 